# Assignment: Extracting Static WebPage

Extract information about “วันพระ” for 3 years from:
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx
- https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx


Note that you can use dateparse package to parse Thai date.  First, we will have to install the package, this is for Google Colab users.  Otherwise, installing via command line is recommended (pip or conda).

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install dateparser

In [ ]:
import dateparser

To convert from thai text date string, we will use the parse method.  Note that the parse method assumes the BC year, not BE.  Thus, we will have to subtract 543 from the year.  In addition, weekday() returns day of week with 0=Monday, ..., 6=Sunday.

In [ ]:
dt = dateparser.parse('วันศุกร์ที่ 17 มกราคม 2563')

# 0: Monday, 1:Tuesday and so on..
print(dt)
print(dt.weekday())

dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2563-01-17 00:00:00
0
2020-01-17 00:00:00
4


In [ ]:
dt = dateparser.parse('วันเสาร์ที่ 21 กันยายน 2564')
dt = dt.replace(year=dt.year-543)
print(dt)
print(dt.weekday())

2021-09-21 00:00:00
1


Count the distribution of number of week days that are “วันพระ” for all three years and answer the following questions:

# Web Scraping


In [ ]:
import requests
from bs4 import BeautifulSoup

thai_days_of_week = [
    "วันจันทร์ที่", "วันอังคารที่", "วันพุธที่",
    "วันพฤหัสบดีที่", "วันศุกร์ที่", "วันเสาร์ที่", "วันอาทิตย์ที่"
]

def extract_dates(url, date_counts_glob):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    elements = soup.find_all('div', {'class': 'bud-day-col'})
    # Initialize a map with Thai days of the week as keys and 0 as initial count
    date_counts = {day: 0 for day in thai_days_of_week}
    sum_date = 0
    for element in elements:
        date_str = element.text.strip()
        # Increment the count for the corresponding day of the week
        for day in thai_days_of_week:
            if date_str.startswith(day):
                date_counts[day] += 1
                date_counts_glob[day] += 1
                sum_date += 1
                break
    return date_counts, sum_date

In [ ]:
for i in thai_days_of_week:
  print(i)

วันจันทร์ที่
วันอังคารที่
วันพุธที่
วันพฤหัสบดีที่
วันศุกร์ที่
วันเสาร์ที่
วันอาทิตย์ที่


In [ ]:
# URLs to scrape
urls = [
    "https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2565.aspx",
    "https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2566.aspx",
    "https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.2567.aspx"
]

In [ ]:
date_counts_glob = {day: 0 for day in thai_days_of_week}
dates_2565, sum_date_2565 = extract_dates(urls[0], date_counts_glob)
dates_2566, sum_date_2566 = extract_dates(urls[1], date_counts_glob)
dates_2567, sum_date_2567 = extract_dates(urls[2], date_counts_glob)

## How many วันพระ in total (of 3 years)?

In [ ]:
print("Total dates:", sum_date_2565 + sum_date_2566 + sum_date_2567)

Total dates: 152


## How many days in total (of 3 years) that วันพระ is Monday?

In [ ]:
print(dates_2565)
print(dates_2566)
print(dates_2567)
print(date_counts_glob)
print("total วันพระ that is monday:", dates_2565['วันจันทร์ที่'] + dates_2566['วันจันทร์ที่'] + dates_2567['วันจันทร์ที่'])

{'วันจันทร์ที่': 7, 'วันอังคารที่': 8, 'วันพุธที่': 8, 'วันพฤหัสบดีที่': 9, 'วันศุกร์ที่': 7, 'วันเสาร์ที่': 6, 'วันอาทิตย์ที่': 6}
{'วันจันทร์ที่': 8, 'วันอังคารที่': 6, 'วันพุธที่': 8, 'วันพฤหัสบดีที่': 6, 'วันศุกร์ที่': 6, 'วันเสาร์ที่': 7, 'วันอาทิตย์ที่': 9}
{'วันจันทร์ที่': 6, 'วันอังคารที่': 6, 'วันพุธที่': 6, 'วันพฤหัสบดีที่': 8, 'วันศุกร์ที่': 8, 'วันเสาร์ที่': 8, 'วันอาทิตย์ที่': 9}
{'วันจันทร์ที่': 21, 'วันอังคารที่': 20, 'วันพุธที่': 22, 'วันพฤหัสบดีที่': 23, 'วันศุกร์ที่': 21, 'วันเสาร์ที่': 21, 'วันอาทิตย์ที่': 24}
total วันพระ that is monday: 21


## Which day of the week that has the minimum number of วันพระ?

In [ ]:
# Find the day with the minimum number of dates
min_day = min(date_counts_glob, key=date_counts_glob.get)
min_count = date_counts_glob[min_day]

In [ ]:
print(f"Day with the minimum number of dates: {min_day} with {min_count} dates")

Day with the minimum number of dates: วันอังคารที่ with 20 dates


## Which day of the week that has the maximum number of วันพระ?

In [ ]:
# Find the day with the maximum number of dates
max_day = max(date_counts_glob, key=date_counts_glob.get)
max_count = date_counts_glob[max_day]

In [ ]:
print(f"Day with the maximum number of dates: {max_day} with {max_count} dates")

Day with the maximum number of dates: วันอาทิตย์ที่ with 24 dates
